# 使用gcc编译扩展模块

Windows下的Python使用Visual Studio编译，编译扩展模块时也推荐使用Visual Studio。但是由于Visual Studio的体积庞大，并且需要安装，使用起来极其不便。本文介绍如何使用Windows下的gcc编译器`mingw32`编译Python的扩展模块。

下面的`setup_test()`创建Cython文件`test_compile.pyx`，并将其编译为扩展模块。

In [1]:
def setup_test(*argv):
    import sys
    sys.argv = ["setup.py"] + list(argv) + ["--force"]
    from distutils.core import setup
    from Cython.Build import cythonize

    with open("test_compile.pyx", "w") as f:
        f.write("""
def add(a, b):
    return a + b
        """
        )
    return setup(
        ext_modules = cythonize("test_compile.pyx")
    )

下面调用`setup_test()`编译。由输出可以看出这里使用的是`Visual Studio`编译。

In [3]:
dist = setup_test("build_ext", "--inplace", "--compiler", "msvc")

Compiling test_compile.pyx because it changed.
[1/1] Cythonizing test_compile.pyx
running build_ext
building 'test_compile' extension
C:\Program Files (x86)\Microsoft Visual Studio 14.0\VC\BIN\x86_amd64\cl.exe /c /nologo /Ox /W3 /GL /DNDEBUG /MD -IC:\Anaconda3\include -IC:\Anaconda3\include "-IC:\Program Files (x86)\Microsoft Visual Studio 14.0\VC\INCLUDE" "-IC:\Program Files (x86)\Microsoft Visual Studio 14.0\VC\ATLMFC\INCLUDE" "-IC:\Program Files (x86)\Windows Kits\10\include\10.0.15063.0\ucrt" "-IC:\Program Files (x86)\Windows Kits\NETFXSDK\4.6.1\include\um" "-IC:\Program Files (x86)\Windows Kits\8.1\include\\shared" "-IC:\Program Files (x86)\Windows Kits\8.1\include\\um" "-IC:\Program Files (x86)\Windows Kits\8.1\include\\winrt" /Tctest_compile.c /Fobuild\temp.win-amd64-3.5\Release\test_compile.obj
C:\Program Files (x86)\Microsoft Visual Studio 14.0\VC\BIN\x86_amd64\link.exe /nologo /INCREMENTAL:NO /LTCG /DLL /MANIFEST:EMBED,ID=2 /MANIFESTUAC:NO /LIBPATH:C:\Anaconda3\libs /LIBPATH:

可以通过`dist.command_options`查看各种选项：

In [5]:
dist.command_options

{'build': {'compiler': ('C:\\Anaconda3\\lib\\distutils\\distutils.cfg',
   'mingw32')},
 'build_ext': {'compiler': ('command line', 'msvc'),
  'force': ('command line', 1),
  'inplace': ('command line', 1)}}

这里`build`的`compiler`选项的值为`'mingw32'`，该值是从`'C:\\Anaconda3\\lib\\distutils\\distutils.cfg'`文件读取的。`build_ext`的`compiler`选项的值为`'msvc'`，它是从命令行参数`sys.argv`中读取的。下面通过添加`--compiler mingw32`命令行参数，指定使用`gcc`编译，但是会出现错误。

In [6]:
dist = setup_test("build_ext", "--inplace", "--compiler", "mingw32")

running build_ext


ValueError: Unknown MS Compiler version 1900 

## 给`cygwinccompiler`模块打补丁

如果使用`gcc`编译失败，则需要给`cygwinccompiler`模块打补丁。一共有三处需要修改的地方，下面先找到该模块对应的位置：

In [2]:
from distutils import cygwinccompiler
print(cygwinccompiler.__file__)

C:\Anaconda3\lib\distutils\cygwinccompiler.py


第一处修改：添加`msc_ver`为`'1700', '1800', '1900'`的判断：

```python
def get_msvcr():
    msc_pos = sys.version.find('MSC v.')
    if msc_pos != -1:
        msc_ver = sys.version[msc_pos+6:msc_pos+10]
        if msc_ver == '1300':
            # MSVC 7.0
            return ['msvcr70']
        elif msc_ver == '1310':
            # MSVC 7.1
            return ['msvcr71']
        elif msc_ver == '1400':
            # VS2005 / MSVC 8.0
            return ['msvcr80']
        elif msc_ver == '1500':
            # VS2008 / MSVC 9.0
            return ['msvcr90']
        elif msc_ver == '1600':
            # VS2010 / MSVC 10.0
            return ['msvcr100']
        elif msc_ver == '1700': #* Add new MSVC versions *#
            # Visual Studio 2012 / Visual C++ 11.0
            return ['msvcr110']
        elif msc_ver == '1800':
            # Visual Studio 2013 / Visual C++ 12.0
            return ['msvcr120']
        elif msc_ver == '1900':
            return ['msvcr140'] #实际上该文件不存在, Anaconda会提供一个Dummy文件
        else:
            raise ValueError("Unknown MS Compiler version %s " % msc_ver)
```

第二处修改`'LIBRARY %s'`改为`'LIBRARY "%s"'`:

```python
def link(self, target_desc, objects, output_filename, output_dir=None,
         libraries=None, library_dirs=None, runtime_library_dirs=None,
         export_symbols=None, debug=0, extra_preargs=None,
         extra_postargs=None, build_temp=None, target_lang=None):
    
        # ...
        # Generate .def file
        contents = [
            'LIBRARY "%s"' % os.path.basename(output_filename),  #* add quote to filename *#
            "EXPORTS"]
        # ...

第三处修改，为`check_outut()`添加`shell=True`参数

def is_cygwingcc():
    out_string = check_output(['gcc', '-dumpmachine'], shell=True) #* add shell=True *#
    return out_string.strip().endswith(b'cygwin')
```

另外还需要安装`libpython`，如果没有安装，执行下面的命令:

```
conda install libpython
```

In [3]:
import distutilspatch

In [4]:
dist = setup_test("build_ext", "--inplace", "--compiler", "mingw32")

Compiling test_compile.pyx because it changed.
[1/1] Cythonizing test_compile.pyx
running build_ext
building 'test_compile' extension
C:\Anaconda3\Scripts\gcc.bat -mdll -O -Wall -IC:\Anaconda3\include -IC:\Anaconda3\include -c test_compile.c -o build\temp.win-amd64-3.5\Release\test_compile.o
writing build\temp.win-amd64-3.5\Release\test_compile.cp35-win_amd64.def
C:\Anaconda3\Scripts\gcc.bat -shared -s build\temp.win-amd64-3.5\Release\test_compile.o build\temp.win-amd64-3.5\Release\test_compile.cp35-win_amd64.def -LC:\Anaconda3\libs -LC:\Anaconda3\PCbuild\amd64 -lpython35 -lmsvcr140 -o C:\Users\RY\Documents\notebooks\cooknotebook\notebooks\python\test_compile.cp35-win_amd64.pyd


## `DISTUTILS_DEBUG`环境变量

如果定义了`DISTUTILS_DEBUG`环境变量，那么编译时会输出调试信息，可以清楚的看出哪些选项来自哪些地方。下面的程序修改了环境变量之后，调用`imp.reload()`重新载入相关模块，让这些模块重新读入环境变量。

In [5]:
import os
os.environ["DISTUTILS_DEBUG"] = "1"
import imp, distutils
imp.reload(distutils.debug)
imp.reload(distutils.core)
dist = setup_test("build_ext", "--inplace", "--compiler", "mingw32")

options (after parsing config files):
option dict for 'build' command:
  {'compiler': ('C:\\Anaconda3\\lib\\distutils\\distutils.cfg', 'mingw32')}
options (after parsing command line):
option dict for 'build' command:
  {'compiler': ('C:\\Anaconda3\\lib\\distutils\\distutils.cfg', 'mingw32')}
option dict for 'build_ext' command:
  {'compiler': ('command line', 'mingw32'),
   'force': ('command line', 1),
   'inplace': ('command line', 1)}
running build_ext
Python's GCC status: ok (details: 'C:\Anaconda3\include\pyconfig.h' mentions '__GNUC__')
mingw32: gcc 4.7.0, ld 2.22.51.20111217, dllwrap 2.22.51.20111217

building 'test_compile' extension
C:\Anaconda3\Scripts\gcc.bat -mdll -O -Wall -IC:\Anaconda3\include -IC:\Anaconda3\include -c test_compile.c -o build\temp.win-amd64-3.5\Release\test_compile.o
writing build\temp.win-amd64-3.5\Release\test_compile.cp35-win_amd64.def
C:\Anaconda3\Scripts\gcc.bat -shared -s build\temp.win-amd64-3.5\Release\test_compile.o build\temp.win-amd64-3.5\Rele

配置文件的优先顺序：

* `distutils`模块所在的路径下的`distutils.cfg`文件。
* 用户目录下的`pydistutils.cfg`文件。
* 当前路径下的`setup.cfg`文件。

下面用程序计算这三个文件的路径：

In [7]:
import sys
from pathlib import Path
print(Path(sys.modules['distutils'].__file__).parent / "distutils.cfg")
print(Path("~/pydistutils.cfg").expanduser())
print(Path("./setup.cfg").absolute())

C:\Anaconda3\lib\distutils\distutils.cfg
C:\Users\RY\pydistutils.cfg
C:\Users\RY\Documents\notebooks\cooknotebook\notebooks\python\setup.cfg


在当前路径下添加`setup.cfg`文件

In [8]:
%%file setup.cfg
[build]
compiler=mingw32

Writing setup.cfg


再次编译发现`build`的`compiler`已经变成了`mingw32`了。

In [9]:
dist = setup_test("build_ext", "--inplace")

options (after parsing config files):
option dict for 'build' command:
  {'compiler': ('setup.cfg', 'mingw32')}
options (after parsing command line):
option dict for 'build' command:
  {'compiler': ('setup.cfg', 'mingw32')}
option dict for 'build_ext' command:
  {'force': ('command line', 1), 'inplace': ('command line', 1)}
running build_ext
Python's GCC status: ok (details: 'C:\Anaconda3\include\pyconfig.h' mentions '__GNUC__')
mingw32: gcc 4.7.0, ld 2.22.51.20111217, dllwrap 2.22.51.20111217

building 'test_compile' extension
C:\Anaconda3\Scripts\gcc.bat -mdll -O -Wall -IC:\Anaconda3\include -IC:\Anaconda3\include -c test_compile.c -o build\temp.win-amd64-3.5\Release\test_compile.o
writing build\temp.win-amd64-3.5\Release\test_compile.cp35-win_amd64.def
C:\Anaconda3\Scripts\gcc.bat -shared -s build\temp.win-amd64-3.5\Release\test_compile.o build\temp.win-amd64-3.5\Release\test_compile.cp35-win_amd64.def -LC:\Anaconda3\libs -LC:\Anaconda3\PCbuild\amd64 -lpython35 -lmsvcr140 -o C:\User